In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# this notebook runs inference for evaluation using the dfci teacher model on the dfci test set

In [3]:
reports = pd.read_csv('/mnt/d/Dropbox (Partners HealthCare)/profile_3-2023/derived_data/labeled_medonc_prissmm_mixedisprog.csv')
reports = reports[reports.split=='test']
inference_input = reports
inference_input['text'] = inference_input['text'].str.lower().str.replace("\n", " ")
inference_input.drop(inference_input.filter(regex='Unnamed|outcome').columns, axis=1, inplace=True)


In [4]:

from transformers import AutoModel



from torch.nn import functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn import LSTM, Linear, Embedding, Conv1d, MaxPool1d, GRU, LSTMCell, Dropout, Module, Sequential, ReLU

   
class LabeledModel(nn.Module):

    def __init__(self):
        super(LabeledModel, self).__init__()
        
        self.longformer = AutoModel.from_pretrained('yikuan8/Clinical-Longformer')
        
        self.any_cancer_head = Sequential(Linear(768, 128), ReLU(), Linear(128,1))
        self.response_head = Sequential(Linear(768, 128), ReLU(), Linear(128,1))
        self.progression_head = Sequential(Linear(768, 128), ReLU(), Linear(128,1))


        
    def forward(self, x_text_tensor, x_attention_mask):
        # x should be tuple of input IDs, then attention mask
        global_attention_mask = torch.zeros_like(x_text_tensor).to('cuda')
        # global attention on cls token
        global_attention_mask[:, 0] = 1
        main = self.longformer(x_text_tensor, x_attention_mask, global_attention_mask)
        main = main.last_hidden_state[:,0,:].squeeze(1)

                                          
        any_cancer_out = self.any_cancer_head(main)
        response_out = self.response_head(main)
        progression_out = self.progression_head(main)



        
        return any_cancer_out, response_out, progression_out
        




In [6]:
from torch.utils import data

class UnLabeledDataset(data.Dataset):
    def __init__(self, pandas_dataset):
        self.data = pandas_dataset.copy()
        self.indices = self.data.index.unique()
        self.tokenizer = AutoTokenizer.from_pretrained("yikuan8/Clinical-Longformer", truncation_side='left')        
        
        
    def __len__(self):
        # how many notes in the dataset
        return len(self.indices)
    
    def __getitem__(self, index):
        # get data for notes corresponding to indices passed
        this_index = self.indices[index]
        pand = self.data.loc[this_index, :]
    
        encoded = self.tokenizer(pand['text'], padding='max_length', truncation=True)

        x_text_tensor = torch.tensor(encoded.input_ids, dtype=torch.long)
        x_attention_mask = torch.tensor(encoded.attention_mask, dtype=torch.long)
        

        return x_text_tensor, x_attention_mask
        
        

In [8]:
# write out inference dataset
themodel = LabeledModel()
themodel.load_state_dict(torch.load('dfci_phi_note_longformer.pt'))
themodel.to('cuda')

themodel.eval()

no_shuffle_valid_dataset = data.DataLoader(UnLabeledDataset(inference_input), batch_size=2, shuffle=False, num_workers=0)

output_prediction_lists = [[] for x in range(3)]
for batch in no_shuffle_valid_dataset:
    x_text_ids = batch[0].to('cuda')
    x_attention_mask = batch[1].to('cuda')
    with torch.no_grad():
        predictions = themodel(x_text_ids, x_attention_mask)
    for j in range(3):
        output_prediction_lists[j].append(predictions[j].detach().cpu().numpy())

output_prediction_lists = [np.concatenate(x) for x in output_prediction_lists]

In [14]:
output_dataset = inference_input.copy()
for x in range(3):
    output_dataset['outcome_' + str(x) + '_logit'] = output_prediction_lists[x]

In [15]:
output_dataset = output_dataset.rename(columns={'outcome_0_logit':'any_cancer_logit',
                                                  'outcome_1_logit':'response_logit',
                                                  'outcome_2_logit':'progression_logit'})


In [16]:
from utils_102023 import eval_model

In [17]:
for outcome in ['any_cancer','progression','response']:
    print('all cancers')
    print(outcome)
    print(eval_model(output_dataset[outcome + '_logit'], output_dataset[outcome], graph=False))
    print("\n")

all cancers
any_cancer
AUC 0.9531537514506728
Outcome probability: 0.8024837374334713
Average precision score: 0.98
Best F1: 0.9624197983501376
Best F1 threshold: -0.049199514
-0.049199514


all cancers
progression
AUC 0.961429361408972
Outcome probability: 0.17060910703725607
Average precision score: 0.85
Best F1: 0.7908937605396289
Best F1 threshold: -0.30774012
-0.30774012


all cancers
response
AUC 0.9714942128333686
Outcome probability: 0.11975162625665287
Average precision score: 0.85
Best F1: 0.7995169082125605
Best F1 threshold: 0.34003437
0.34003437




In [ ]:
# fix nsclc below (has both phase 2s)

In [19]:
output_dataset['cancer_type'] = np.where(output_dataset.cancer_type.str.contains('nsclc'), 'nsclc', output_dataset.cancer_type)

In [21]:
for cancer in output_dataset.cancer_type.unique():
    subset = output_dataset[output_dataset.cancer_type == cancer]
    for outcome in ['any_cancer','progression','response']:
        print(cancer)
        print(outcome)
        print(eval_model(subset[outcome + '_logit'], subset[outcome], graph=False))
        print("\n")
    

prostate
any_cancer
AUC 0.9451417156641037
Outcome probability: 0.7528089887640449
Average precision score: 0.98
Best F1: 0.927070457354759
Best F1 threshold: 0.22990912
0.22990912


prostate
progression
AUC 0.9551820728291317
Outcome probability: 0.09550561797752809
Average precision score: 0.73
Best F1: 0.7090909090909091
Best F1 threshold: -0.5175541
-0.5175541


prostate
response
AUC 0.9509147701918785
Outcome probability: 0.06741573033707865
Average precision score: 0.59
Best F1: 0.676056338028169
Best F1 threshold: 0.41871846
0.41871846


breast
any_cancer
AUC 0.9928493203633318
Outcome probability: 0.8935643564356436
Average precision score: 1.00
Best F1: 0.9862637362637363
Best F1 threshold: -3.667033
-3.667033


breast
progression
AUC 0.9825996353216984
Outcome probability: 0.13613861386138615
Average precision score: 0.93
Best F1: 0.8571428571428571
Best F1 threshold: -1.1806877
-1.1806877


breast
response
AUC 0.9709653647752394
Outcome probability: 0.14603960396039603
Avera

/mnt/d/Dropbox (Partners HealthCare)/impression_bert/prissmm_medonc/utils_102023.py:111: RuntimeWarning: invalid value encountered in divide
  F1 = 2*((precision*recall)/(precision+recall))
